<a href="https://colab.research.google.com/github/MehmetSemerci/CNG562-MehmetSemerci/blob/master/SEMERCI_otto_group_product_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

url = 'https://raw.githubusercontent.com/MehmetSemerci/CNG562-MehmetSemerci/master/OttoTrain.csv'
df = pd.read_csv(url)

#
df[df.columns[94]]=df[df.columns[94]].replace("Class_1",1)
df[df.columns[94]]=df[df.columns[94]].replace("Class_2",2)
df[df.columns[94]]=df[df.columns[94]].replace("Class_3",3)
df[df.columns[94]]=df[df.columns[94]].replace("Class_4",4)
df[df.columns[94]]=df[df.columns[94]].replace("Class_5",5)
df[df.columns[94]]=df[df.columns[94]].replace("Class_6",6)
df[df.columns[94]]=df[df.columns[94]].replace("Class_7",7)
df[df.columns[94]]=df[df.columns[94]].replace("Class_8",8)
df[df.columns[94]]=df[df.columns[94]].replace("Class_9",9)


x = df[df.columns[1:94]].values
y = df[df.columns[94]]


# Multi-layer Perceptron (MLP)

In [4]:
from sklearn.neural_network import MLPClassifier

x_train, x_test, y_train, y_test = train_test_split(x,y, test_size= 0.20, random_state=27)

clf = MLPClassifier(hidden_layer_sizes=(70,35,15), max_iter=300, alpha=0.0001,
                     solver='sgd',  random_state=21,tol=0.000000001)


clf.fit(x_train, y_train)
y_pred = clf.predict(x_test)


/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


In [5]:
from sklearn.metrics import accuracy_score

accuracy_score(y_test, y_pred)

0.7916936005171299

#KNN

In [6]:
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier(n_neighbors=5)

knn.fit(x_train, y_train)

predictknn = knn.predict(x_test)

accuracy_score(y_test,predictknn)

0.7740788623141565

# SVM

In [8]:
from sklearn.svm import SVC  

svclassifier = SVC(kernel='linear')  

svclassifier.fit(x_train, y_train)  

y_pred = svclassifier.predict(x_test)  

accuracy_score(y_test, y_pred)

0.7694731738849386

#Random Forest

#XGBoost

In [13]:

from xgboost import XGBClassifier


model = XGBClassifier()
model.fit(x_train, y_train)


y_pred = model.predict(x_test)
accuracy_score(y_test, y_pred)

0.7703619909502263